<a href="https://colab.research.google.com/github/n1xd/5900/blob/main/CIT5900_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# setup
!pip install requests beautifulsoup4 lxml pandas

# Web Scraping

In [ ]:
# PI, year, title, DOI/unique identifier

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json
from urllib.parse import quote_plus, unquote
import random

In [ ]:
# load PIs from excel file (helper)
def load_excel_data(file_path, sheet_name="All Metadata"):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        print(f"Loaded {len(df)} records from Excel")
        return df
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None

In [ ]:
# load PIs (run)
excel_path = "projects_all_metadata.xlsx"
df = load_excel_data(excel_path)

# show selected
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)
print(df[['Proj ID', 'Title', 'Start Year', 'End Year', 'PI']].head())

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# load metadata authors
authors = df['PI'].dropna().unique().tolist()  # limiting to 100 for testing

# define keywords - convert to lowercase for case-insensitive matching
fsrdc_keywords = [
    'fsrdc', 'census bureau', 'restricted data', 'restricted microdata',
    'disclosure review', 'confidentiality review',
    'annual survey of manufactures', 'census of manufactures',
    'michigan rdc', 'irs', 'bea', 'federal statistical'
]
fsrdc_keywords = [keyword.lower() for keyword in fsrdc_keywords]

# prep list for results (later df)
matched_papers = []

for i, author in enumerate(authors, start=1):
    # Format author name for URL
    formatted_author = author.replace(" ", "+")
    url = f"https://arxiv.org/search/?query={formatted_author}&searchtype=author"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        soup = BeautifulSoup(response.text, "html.parser")
        results = soup.find_all('li', class_='arxiv-result')

        print(f"Search {i}/{len(authors)}: {author} - Found {len(results)} papers")

        for r in results[:10]:  # check up to 10 papers per author
            try:
                title = r.find('p', class_='title').text.strip()
                abs_link_element = r.find('p', class_='list-title').find('a')
                if abs_link_element and 'href' in abs_link_element.attrs:
                    abs_link = abs_link_element['href']
                    if not abs_link.startswith('http'):
                        abs_link = f"https://arxiv.org{abs_link}"

                    print(f"checking paper: {title[:50]}...")
                    print(f"link: {abs_link}")

                    # parse abstract
                    abs_response = requests.get(abs_link)
                    abs_soup = BeautifulSoup(abs_response.text, "html.parser")
                    abstract_element = abs_soup.find('blockquote', class_='abstract')

                    if abstract_element:
                        # Normalize abstract: remove extra spaces, convert to lowercase
                        raw_abstract = abstract_element.text.strip()
                        abstract = re.sub(r'Abstract: ', '', raw_abstract, flags=re.IGNORECASE)
                        abstract = re.sub(r'\s+', ' ', abstract).lower()  # Normalize whitespace and lowercase

                        print(f"abstract snippet: {abstract[:100]}...")

                        # Also check the title for keywords
                        normalized_title = title.lower()
                        combined_text = f"{normalized_title} {abstract}"

                        # check for keyword match in abstract and title
                        found_keywords = []
                        for keyword in fsrdc_keywords:
                            # For single-word keywords, use word boundaries
                            if ' ' not in keyword:
                                pattern = rf"\b{re.escape(keyword)}\b"
                                if re.search(pattern, combined_text):
                                    found_keywords.append(keyword)
                                    print(f"Found keyword: '{keyword}' as whole word")
                            # For multi-word keywords, try partial word matching but ensure each word is a standalone word
                            else:
                                words = keyword.split()
                                # Check if all words in the multi-word keyword exist as whole words
                                if all(re.search(rf"\b{re.escape(word)}\b", combined_text) for word in words):
                                    found_keywords.append(keyword)
                                    print(f"Found multi-word keyword: '{keyword}'")

                        if found_keywords:  # Only process if keywords were found
                            print(f"keywords found: {', '.join(found_keywords)}")

                            # get year + doi (if possible, will filter later)
                            history = abs_soup.find('div', class_='submission-history')
                            year = None
                            if history:
                                match = re.search(r'\b(20\d{2})\b', history.text)
                                year = match.group(0) if match else None
                            else:
                                year = None

                            doi_element = abs_soup.find('td', class_='tablecell doi')
                            doi = doi_element.text.strip() if doi_element else None

                            matched_papers.append({
                                "pi": author,
                                "title": title,
                                "year": year,
                                "doi": doi if doi else "N/A",
                                "abstract": raw_abstract,  # Store original abstract
                                "keywords_found": ", ".join(found_keywords),
                                "url": abs_link
                            })
                        else:
                            print(f"no keywords matched in paper")
                    else:
                        print(f"no abstract")
            except Exception as e:
                print(f"error processing paper: {str(e)}")

        time.sleep(3)  # gentle scraping
    except Exception as e:
        print(f"error searching for author {author}: {str(e)}")

# save
if matched_papers:
    matched_df = pd.DataFrame(matched_papers)
    matched_df.to_csv("fsrdc_keyword_matches.csv", index=False)
    print(f"{len(matched_df)} matching papers saved")
else:
    print("no matches found")

In [ ]:
print(matched_df.head(10))

# Some API

In [ ]:
# configuration
MAX_RETRIES = 3
DELAY_BETWEEN_REQUESTS = 2  # seconds
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
]

In [ ]:
# load PIs from excel file (helper)
def load_excel_data(file_path, sheet_name="All Metadata"):
    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        print(f"Loaded {len(df)} records from Excel")
        return df
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None

In [ ]:
# load PIs (run)
excel_path = "projects_all_metadata.xlsx"
df = load_excel_data(excel_path)

# show selected
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 100)
print(df[['Proj ID', 'Title', 'Start Year', 'End Year', 'PI']].head())

In [ ]:
# API crossref function
def search_scholarly_articles(pi_name, limit=10):
    """Search for scholarly articles by PI name using Crossref API."""
    base_url = "https://api.crossref.org/works"
    params = {
        "query.author": pi_name,
        "rows": limit,
        "sort": "relevance",
        "order": "desc"
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            results = []

            for item in data.get("message", {}).get("items", []):
                # Extract metadata
                title = item.get("title", [""])[0] if item.get("title") else ""

                # Extract authors
                authors = []
                for author in item.get("author", []):
                    if "given" in author and "family" in author:
                        authors.append(f"{author['given']} {author['family']}")

                # Extract year
                year = None
                if "published-print" in item and "date-parts" in item["published-print"]:
                    year = item["published-print"]["date-parts"][0][0]
                elif "published-online" in item and "date-parts" in item["published-online"]:
                    year = item["published-online"]["date-parts"][0][0]

                # Extract DOI
                doi = item.get("DOI", "")

                # Extract URL
                url = f"https://doi.org/{doi}" if doi else ""

                results.append({
                    "title": title,
                    "authors": authors,
                    "year": year,
                    "doi": doi,
                    "url": url
                })

            return results
        else:
            print(f"Error searching Crossref API: Status code {response.status_code}")
            return []
    except Exception as e:
        print(f"Exception when searching scholarly articles: {e}")
        return []

In [ ]:
# test API cross
if df is not None and len(df) > 0:
    sample_pi = df['PI'].iloc[0]
    print(f"Testing Crossref API with sample PI: {sample_pi}")
    sample_results = search_scholarly_articles(sample_pi)
    print(f"Found {len(sample_results)} papers for {sample_pi}")
    if sample_results:
        for i, paper in enumerate(sample_results):  # Show first 3 results
            print(f"\nPaper {i+1}:")
            print(f"Title: {paper['title']}")
            print(f"Authors: {', '.join(paper['authors'])}")
            print(f"Year: {paper['year']}")
            print(f"DOI: {paper['doi']}")

In [ ]:
fsrdc_keywords = [
    'fsrdc', 'census bureau', 'restricted data', 'restricted microdata',
    'disclosure review', 'confidentiality review',
    'annual survey of manufactures', 'census of manufactures',
    'michigan rdc', 'irs', 'bea'
]

def fetch_abstract_from_doi(doi):
    if not doi:
        return None

    url = f"https://api.crossref.org/works/{doi}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            item = response.json().get("message", {})
            abstract = item.get("abstract", None)
            if abstract:
                # Strip HTML if present
                return BeautifulSoup(abstract, "html.parser").get_text()
    except Exception as e:
        print(f"Error fetching abstract for DOI {doi}: {e}")
    return None

def score_fsrdc_in_abstract(abstract, keywords=fsrdc_keywords):
    if not abstract:
        return 0
    text = abstract.lower()
    return sum(1 for kw in keywords if kw in text)

In [ ]:
def process_pis_for_papers(df, limit_pis=200):
    all_papers = []
    unique_pis = df['PI'].dropna().unique()[:limit_pis]

    print(f"🔍 Searching papers for {len(unique_pis)} PIs...")

    for i, pi_name in enumerate(unique_pis):
        print(f"\nPI {i+1}/{len(unique_pis)}: {pi_name}")
        papers = search_scholarly_articles(pi_name, limit=5)

        for paper in papers:
            paper['pi'] = pi_name
            abstract = fetch_abstract_from_doi(paper['doi'])
            paper['abstract'] = abstract
            paper['fsrdc_score'] = score_fsrdc_in_abstract(abstract)

        all_papers.extend(papers)
        time.sleep(DELAY_BETWEEN_REQUESTS)

    return pd.DataFrame(all_papers)

In [ ]:
# see sample
sample_df = process_pis_for_papers(df)

sorted_df = sample_df.sort_values(by='fsrdc_score', ascending=False)

In [ ]:
fsrdc_related = sample_df[sample_df['fsrdc_score'] > 0]
cleaned_df = fsrdc_related.dropna(subset=['title', 'authors', 'year']).copy() # drop rows that miss year
cleaned_df['year'] = cleaned_df['year'].astype(int) # convert years to int
cleaned_df['title'] = cleaned_df['title'].str.lower() # lowercase titles
cleaned_df = cleaned_df[cleaned_df['year'] != 2024] # get rid of 2024 papers
cleaned_df = cleaned_df.sort_values(by='fsrdc_score', ascending=False) # sort by score

# Show the result
print(len(cleaned_df), "papers found")
print(cleaned_df[['title', 'authors', 'year', 'doi', 'fsrdc_score']].head(10))
cleaned_df.to_csv("fsrdc_related_papers.csv", index=False)

# More API

Extracting PIs from the excel data

In [ ]:
import pandas as pd

def extract_unique_pis_from_excel(file_path, sheet_name, possible_pi_columns=['PI', 'Principal Investigator', 'PI Name']):
    """
    Extract all unique Principal Investigators (PIs) from an Excel file.

    Args:
        file_path (str): Path to the Excel file
        sheet_name (str/int): Name or index of sheet to read
        possible_pi_columns (list): Possible column names that might contain PI information

    Returns:
        list: List of unique PI names
    """
    try:
        # Read the Excel file
        df = pd.read_excel(file_path, sheet_name=sheet_name)

        # Find which column contains PI information
        pi_column = None
        for col in possible_pi_columns:
            if col in df.columns:
                pi_column = col
                break

        if not pi_column:
            available_columns = df.columns.tolist()
            raise ValueError(
                f"Could not find PI column. Tried: {possible_pi_columns}\n"
                f"Available columns: {available_columns}"
            )

        # Extract unique PIs with robust cleaning
        unique_pis = (
            df[pi_column]
            .dropna()  # Remove NaN values
            .astype(str)  # Convert to string
            .str.strip()  # Remove whitespace
            .replace('', pd.NA)  # Treat empty strings as NA
            .dropna()  # Remove NA values again
            .unique()  # Get unique values
            .tolist()  # Convert to list
        )

        print(f"Found {len(unique_pis)} unique PIs in column '{pi_column}'")
        return unique_pis

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return []
    except Exception as e:
        print(f"An error occurred while reading the Excel file: {str(e)}")
        return []

# Main execution
if __name__ == "__main__":
    # Path to your Excel file
    excel_file_path = "ProjectsAllMetadata.xlsx"

    # Get all unique PIs from Sheet 2 only
    print("Extracting PIs from Sheet 2 only...")
    unique_pis = extract_unique_pis_from_excel(excel_file_path, sheet_name=1)  # Sheet 2 is index 1

    if unique_pis:
        # Create a DataFrame with the PIs
        pi_df = pd.DataFrame({'Principal Investigator': unique_pis})

        # Save to CSV
        output_file = "unique_PIs.csv"
        pi_df.to_csv(output_file, index=False)
        print(f"\nSaved {len(unique_pis)} unique PIs to {output_file}")

        # Print first 10 as a sample
        print("\nFirst 10 PIs found:")
        for pi in unique_pis[:10]:
            print(pi)
    else:
        print("No PIs found in Sheet 2 of the Excel file")

Orcid API

In [ ]:
import csv
import requests
import pandas as pd
from time import sleep

# ORCID API endpoints
ORCID_SEARCH_URL = "https://pub.orcid.org/v3.0/search/"
ORCID_WORKS_URL = "https://pub.orcid.org/v3.0/{}/works"

# Headers for ORCID API
headers = {
    'Accept': 'application/json',
}

# Criteria for filtering research papers
fsrdc_keywords = [
    "Census Bureau", "FSRDC", "Federal Statistical Research Data Center",
    "restricted microdata", "IRS", "BEA", "confidentiality review",
    "Michigan RDC", "Texas RDC", "Boston RDC",
    "Annual Survey of Manufactures", "Census of Construction Industries",
    "Census of Finance, Insurance, and Real Estate"
]

fsrdc_keywords = [kw.lower() for kw in fsrdc_keywords]  # Convert to lowercase

def contains_fsrdc_keyword(text):
    """Check if text contains any FSRDC keyword"""
    if not text or not isinstance(text, str):
        return False
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in fsrdc_keywords)

def search_orcid_by_name(name):
    """Search ORCID for profiles matching a given name"""
    try:
        if not name or not isinstance(name, str):
            return None

        name_parts = name.strip().split()
        if not name_parts:
            return None

        given_name = name_parts[0]
        family_name = name_parts[-1] if len(name_parts) > 1 else ""

        query = f"given-names:{given_name}"
        if family_name:
            query += f" AND family-name:{family_name}"

        params = {
            'q': query,
            'start': 0,
            'rows': 1
        }

        response = requests.get(ORCID_SEARCH_URL, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            if isinstance(data, dict) and data.get('result'):
                if isinstance(data['result'], list) and len(data['result']) > 0:
                    if isinstance(data['result'][0], dict):
                        return data['result'][0].get('orcid-identifier', {}).get('path')
        return None
    except Exception as e:
        print(f"Search error for {name}: {str(e)}")
        return None

def get_orcid_works(orcid_id):
    """Fetch works for a given ORCID ID"""
    if not orcid_id:
        return None

    try:
        response = requests.get(ORCID_WORKS_URL.format(orcid_id), headers=headers)
        if response.status_code == 200:
            return response.json()
        return None
    except Exception as e:
        print(f"Works error for {orcid_id}: {str(e)}")
        return None

def process_works(pi_name, orcid_id, works_data):
    """Extract publication data from works, filtering for FSRDC keywords"""
    records = []
    if not isinstance(works_data, dict):
        return records

    try:
        for group in works_data.get('group', []):
            for work in group.get('work-summary', []):
                # Get title
                title = work.get('title', {}).get('title', {}).get('value', 'No title')

                # Skip if title doesn't contain FSRDC keywords
                if not contains_fsrdc_keyword(title):
                    continue

                # Get year
                year = work.get('publication-date', {}).get('year', {}).get('value', '')

                # Get DOI
                doi = ''
                for ext_id in work.get('external-ids', {}).get('external-id', []):
                    if ext_id.get('external-id-type', '').lower() == 'doi':
                        doi = ext_id.get('external-id-value', '')
                        break

                # Get journal or venue name if available
                journal = ''
                if 'journal-title' in work:
                    journal = work.get('journal-title', {}).get('value', '')

                records.append({
                    'PI': pi_name,
                    'ORCID_ID': orcid_id,
                    'year': year,
                    'title': title,
                    'DOI': doi,
                    'journal': journal
                })
    except Exception as e:
        print(f"Processing error for {pi_name}: {str(e)}")

    return records

def main():
    # File paths
    input_file = 'unique_pis.csv'
    output_file = 'orcid_api_output.csv'  # Changed output filename

    # Read PI names
    try:
        with open(input_file, mode='r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader, None)  # Skip header
            pi_names = [row[0].strip() for row in reader if row and row[0].strip()]
    except Exception as e:
        print(f"Error reading input: {str(e)}")
        return

    # Process all PIs
    results = []
    total_pis = len(pi_names)

    for i, pi_name in enumerate(pi_names):
        print(f"Processing {i+1}/{total_pis}: {pi_name}")

        # Search ORCID
        orcid_id = search_orcid_by_name(pi_name)
        sleep(1.2)  # Rate limiting

        if not orcid_id:
            # Only include PIs with no ORCID if we're tracking them
            continue

        # Get works
        works_data = get_orcid_works(orcid_id)
        sleep(1.2)  # Rate limiting

        publications = process_works(pi_name, orcid_id, works_data) if works_data else []

        if publications:
            results.extend(publications)

    # Save all results
    if results:
        # Include journal in the output
        pd.DataFrame(results)[['PI', 'year', 'title', 'DOI', 'journal', 'ORCID_ID']] \
          .to_csv(output_file, index=False)
        print(f"\nDone! Saved {len(results)} FSRDC-related records to {output_file}")
    else:
        print("\nNo FSRDC-related publications found")

if __name__ == '__main__':
    main()

CORE API

In [ ]:
import csv
import requests
import pandas as pd
import time
import os
from datetime import datetime

# Configuration
CORE_API_URL = "https://api.core.ac.uk/v3/search/works"
CORE_API_KEY = "JitbKoV5gnCsEOXGWywqeA9zdNcvZSul"  # My API key
INPUT_FILE = 'unique_pis.csv'
OUTPUT_FILE = 'core_api_output.csv'
CHECKPOINT_FILE = 'processing_checkpoint.txt'
LOG_FILE = 'api_processing.log'

# Rate limiting - 100 requests per minute = 0.6 seconds between requests
REQUEST_DELAY = 0.6  # 600ms delay between API calls
MAX_RETRIES = 3  # Max retries for failed requests
RETRY_DELAY = 5  # Initial retry delay in seconds

headers = {
    'Authorization': f'Bearer {CORE_API_KEY}',
    'Accept': 'application/json'
}

# FSRDC keywords (case-insensitive)
fsrdc_keywords = [
    "census bureau", "fsrdc", "federal statistical research data center",
    "restricted microdata", "irs", "bea", "confidentiality review",
    "michigan rdc", "texas rdc", "boston rdc",
    "annual survey of manufactures", "census of construction industries",
    "census of finance, insurance, and real estate"
]

def log_message(message):
    """Log messages with timestamps"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_FILE, 'a') as f:
        f.write(f"[{timestamp}] {message}\n")
    print(message)

def save_checkpoint(index, pi_name):
    """Save current progress"""
    with open(CHECKPOINT_FILE, 'w') as f:
        f.write(f"{index},{pi_name}")

def load_checkpoint():
    """Load last checkpoint if exists"""
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            content = f.read().strip().split(',')
            return int(content[0]), content[1]
    return 0, ""

def safe_lower(text):
    """Safely convert text to lowercase"""
    return text.lower() if text else ""

def is_fsrdc_related(work):
    """Check if paper is FSRDC-related"""
    if not work or not isinstance(work, dict):
        return False

    try:
        # Combine all relevant text fields
        text_fields = [
            safe_lower(work.get('title', '')),
            safe_lower(work.get('abstract', '')),
            safe_lower(work.get('fullText', '')),
            ' '.join([safe_lower(kw) for kw in work.get('keywords', []) if kw]),
            ' '.join([safe_lower(inst.get('name', '')) for inst in work.get('institutions', []) if inst])
        ]
        combined_text = ' '.join(text_fields)
        return any(keyword in combined_text for keyword in fsrdc_keywords)
    except Exception as e:
        log_message(f"Error checking FSRDC relevance: {str(e)}")
        return False

def make_api_request(url, params, retry_count=0):
    """Make API request with rate limit handling"""
    try:
        response = requests.get(url, headers=headers, params=params)

        # Handle rate limiting
        if response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 60))
            log_message(f"Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            return make_api_request(url, params)  # Retry after waiting

        if response.status_code != 200:
            raise Exception(f"HTTP {response.status_code}: {response.text}")

        return response.json()

    except Exception as e:
        if retry_count < MAX_RETRIES:
            wait_time = RETRY_DELAY * (retry_count + 1)
            log_message(f"Attempt {retry_count+1} failed. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
            return make_api_request(url, params, retry_count + 1)
        else:
            log_message(f"Max retries exceeded for request: {str(e)}")
            return None

def search_core_by_author(author_name):
    """Search CORE for works by author with pagination"""
    if not author_name:
        return []

    params = {'q': f'authors:"{author_name}"', 'limit': 100, 'offset': 0}
    all_results = []

    while True:
        data = make_api_request(CORE_API_URL, params)
        if not data:
            break

        results = data.get('results', [])
        fsrdc_results = [r for r in results if is_fsrdc_related(r)]
        all_results.extend(fsrdc_results)

        if len(results) < params['limit']:
            break

        params['offset'] += params['limit']
        time.sleep(REQUEST_DELAY)  # Maintain rate limit between paginated requests

    return all_results

def process_core_results(pi_name, core_data):
    """Process API results into standardized format"""
    records = []
    for work in core_data or []:
        try:
            # Extract publication details
            title = work.get('title', 'No title')
            year = work.get('publishedDate', '')[:4] if work.get('publishedDate') else ''
            doi = work.get('doi', '')

            # Extract institutions
            institutions = ', '.join(filter(None, [
                inst.get('name', '') for inst in work.get('institutions', []) if inst
            ]))

            # Extract matched keyword
            matched_keyword = next(
                (kw for kw in fsrdc_keywords
                 if kw in safe_lower(work.get('title', '') + ' ' + safe_lower(work.get('abstract', '')))),
                ''
            )

            records.append({
                'PI': pi_name,
                'year': year,
                'title': title,
                'DOI': doi,
                'Institutions': institutions,
                'Matched_FSRDC_Keyword': matched_keyword,
                'Keywords': ', '.join(work.get('keywords', [])),
                'Source': 'CORE'
            })
        except Exception as e:
            log_message(f"Error processing work for {pi_name}: {str(e)}")

    return records

def main():
    # Verify API key
    test_response = make_api_request(CORE_API_URL, {'q': 'test', 'limit': 1})
    if not test_response:
        log_message("API connection test failed. Please check your API key.")
        return

    # Load PI names
    try:
        with open(INPUT_FILE, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            next(reader, None)  # Skip header
            all_pi_names = [row[0].strip() for row in reader if row and row[0].strip()]
    except Exception as e:
        log_message(f"Error reading input file: {str(e)}")
        return

    # Load or create results file
    if os.path.exists(OUTPUT_FILE):
        results_df = pd.read_csv(OUTPUT_FILE)
    else:
        results_df = pd.DataFrame(columns=[
            'PI', 'year', 'title', 'DOI', 'Institutions',
            'Matched_FSRDC_Keyword', 'Keywords', 'Source'
        ])

    # Resume from checkpoint
    start_idx, last_pi = load_checkpoint()
    if last_pi:
        log_message(f"Resuming from {last_pi} (index {start_idx})")

    # Process PIs
    for i, pi_name in enumerate(all_pi_names[start_idx:], start=start_idx):
        log_message(f"\nProcessing {i+1}/{len(all_pi_names)}: {pi_name}")

        try:
            # Search and process results
            core_data = search_core_by_author(pi_name)
            publications = process_core_results(pi_name, core_data)

            if publications:
                new_data = pd.DataFrame(publications)
                results_df = pd.concat([results_df, new_data], ignore_index=True)
                log_message(f"Found {len(publications)} FSRDC-related publications")
            else:
                empty_record = pd.DataFrame([{
                    'PI': pi_name,
                    'year': '', 'title': '', 'DOI': '',
                    'Institutions': '', 'Matched_FSRDC_Keyword': '',
                    'Keywords': '', 'Source': 'CORE'
                }])
                results_df = pd.concat([results_df, empty_record], ignore_index=True)
                log_message("No FSRDC-related publications found")

            # Save progress after each PI
            save_checkpoint(i + 1, pi_name)
            results_df.to_csv(OUTPUT_FILE, index=False)
            time.sleep(REQUEST_DELAY)

        except KeyboardInterrupt:
            log_message("\nProcessing interrupted by user. Progress saved.")
            return
        except Exception as e:
            log_message(f"Error processing {pi_name}: {str(e)}")
            continue

    # Clean up after completion
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)
    log_message("\nProcessing complete! Final results saved.")

if __name__ == '__main__':
    main()

Open Alex API


In [ ]:
import pandas as pd
import requests

# Step 1: Load the Excel file from GitHub
url = "https://github.com/dingkaihua/fsrdc-external-census-projects/raw/master/ProjectsAllMetadata.xlsx"
xls = pd.ExcelFile(url)

# Step 2: Read the 'All Metadata' sheet
df_metadata = xls.parse('All Metadata')

# Step 3: Extract unique PI names
pi_names = df_metadata['PI'].dropna().unique()

print("First few PI names:", pi_names[:10])



In [ ]:
#Storing the OpenAlex Ids of the PIs
import time
import requests
import pandas as pd

# OpenAlex API Endpoints
openalex_author_url = "https://api.openalex.org/authors"

# Dictionary to store PI -> OpenAlex ID mappings
pi_openalex_ids = {}

# Find OpenAlex ID for each PI
for pi in pi_names:
    query_name = pi.strip().lower()  # Normalize names
    params = {"search": query_name}

    response = requests.get(openalex_author_url, params=params)
    time.sleep(1)  # Respect API rate limits

    if response.status_code == 200:
        data = response.json()
        if data.get('results'):
            # Pick the first matching author
            author_info = data['results'][0]
            pi_openalex_ids[pi] = author_info['id']
        else:
            pi_openalex_ids[pi] = "Not Found"
    else:
        pi_openalex_ids[pi] = "API Error"

# Convert dictionary to DataFrame and save to CSV
df = pd.DataFrame(list(pi_openalex_ids.items()), columns=["PI Name", "OpenAlex ID"])
df.to_csv("pi_openalex_ids.csv", index=False)

# Final print statement
print(f"\n✅ Total PIs stored: {len(pi_openalex_ids)}")
print("📁 PI OpenAlex IDs saved to 'pi_openalex_ids.csv'")


In [ ]:
#Filtering the research papers of the PIs using FSRDC keywords
import time
import requests
import pandas as pd
from typing import List, Dict
from concurrent.futures import ThreadPoolExecutor, as_completed

# OpenAlex API Endpoints
openalex_works_url = "https://api.openalex.org/works"

# Load PI OpenAlex IDs from CSV
df_pis = pd.read_csv("pi_openalex_ids.csv")
pi_openalex_ids = dict(zip(df_pis["PI Name"], df_pis["OpenAlex ID"]))

# Criteria for filtering research papers
fsrdc_keywords = [
    "Census Bureau", "FSRDC", "Federal Statistical Research Data Center",
    "restricted microdata", "IRS", "BEA", "confidentiality review",
    "Michigan RDC", "Texas RDC", "Boston RDC",
    "Annual Survey of Manufactures", "Census of Construction Industries",
    "Census of Finance, Insurance, and Real Estate"
]

fsrdc_keywords = [kw.lower() for kw in fsrdc_keywords]  # Convert to lowercase

filtered_papers = []
total_pis_processed = 0

def fetch_papers(pi, author_id):
    """Fetch research papers for a given PI asynchronously."""
    if author_id in ["Not Found", "API Error"]:
        return []

    papers = []
    page = 1
    has_more_pages = True

    while has_more_pages:
        params = {
            "filter": f"authorships.author.id:{author_id}",
            "per-page": 200,
            "page": page
        }

        try:
            response = requests.get(openalex_works_url, params=params)

            if response.status_code == 200:
                works_data = response.json()
                results = works_data.get('results', [])

                for work in results:
                    title = work.get('title', 'No Title')
                    abstract = work.get('abstract_inverted_index', {})
                    abstract_text = " ".join(abstract.keys()).lower() if abstract else ""

                    # Check for keyword matches
                    title_lower = title.lower()
                    if any(keyword in title_lower for keyword in fsrdc_keywords) or \
                       any(keyword in abstract_text for keyword in fsrdc_keywords):

                        papers.append({
                            "PI": pi,
                            "ORCID": author_id,
                            "Paper Title": title,
                            "Year": work.get('publication_year', 'Unknown'),
                            "DOI": work.get('doi', 'No DOI')
                        })

                has_more_pages = len(results) >= 50
                page += 1

            elif response.status_code == 429:  # Too many requests (API Rate Limit)
                time.sleep(2)  # Wait and retry
            else:
                break  # Stop processing on error

        except Exception:
            break

    return papers


# Run API calls in parallel using ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:  # 10 concurrent threads
    future_to_pi = {executor.submit(fetch_papers, pi, author_id): pi for pi, author_id in pi_openalex_ids.items()}

    for future in as_completed(future_to_pi):
        pi = future_to_pi[future]
        try:
            papers = future.result()
            filtered_papers.extend(papers)
        except Exception:
            pass  # Ignore errors for now

        total_pis_processed += 1

print(f"\n🔹 Processing complete!")
print(f"   ✅ Total PIs processed: {total_pis_processed}")
print(f"   ✅ Total matched papers: {len(filtered_papers)}")

def save_to_csv(results: List[Dict], filename: str):
    """Save results to CSV file"""
    if not results:
        return

    df = pd.DataFrame(results)
    df = df[["PI", "ORCID", "Paper Title", "Year", "DOI"]]
    df.to_csv(filename, index=False)


# Save to CSV
save_to_csv(filtered_papers, "openalex_api_output.csv")

NIH PMC


In [ ]:
#Storing the PMC Ids of the PIs
import time
import requests
from xml.etree import ElementTree
import pandas as pd

# NIH E-utilities API endpoints
ncbi_esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
ncbi_efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# Dictionary to store PI -> PubMed Central (PMC) IDs
pi_pmc_ids = {}

# Find PMC ID for each PI
for pi in pi_names:
    query_name = pi.strip()  # Normalize names
    params = {
        "db": "pubmed",  # Searching PubMed (PMC articles are indexed here)
        "term": query_name,
        "retmax": 1,  # Limit to one result per query for now
        "retmode": "xml"
    }

    response = requests.get(ncbi_esearch_url, params=params)
    time.sleep(1)  # Respect API rate limits

    if response.status_code == 200:
        # Parse the XML response
        root = ElementTree.fromstring(response.content)
        id_list = root.find("IdList")
        if id_list is not None and len(id_list.findall("Id")) > 0:
            # Retrieve the first PubMed ID for the author
            pmid = id_list.find("Id").text
            pi_pmc_ids[pi] = pmid
    else:
        pass  # Handle errors silently if needed

# Convert dictionary to DataFrame and save to CSV
df = pd.DataFrame(list(pi_pmc_ids.items()), columns=["PI Name", "PMC ID"])
df.to_csv("pi_pmc_ids.csv", index=False)

# Final print statement
print(f"\nTotal PIs stored: {len(pi_pmc_ids)}")
print("PI PMC IDs saved to 'pi_pmc_ids.csv'")


In [ ]:
#Filtering the research papers of the PIs using FSRDC keywords
import time
import requests
import pandas as pd
from xml.etree import ElementTree

# NIH E-utilities API endpoints
ncbi_esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
ncbi_efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

# Criteria for filtering research papers
fsrdc_keywords = [
    "Census Bureau", "FSRDC", "Federal Statistical Research Data Center",
    "restricted microdata", "IRS", "BEA", "confidentiality review",
    "Michigan RDC", "Texas RDC", "Boston RDC",
    "Annual Survey of Manufactures", "Census of Construction Industries",
    "Census of Finance, Insurance, and Real Estate"
]
fsrdc_keywords = [kw.lower() for kw in fsrdc_keywords]

# Load PIs and their PMC IDs from the CSV file
pi_data = pd.read_csv("pi_pmc_ids.csv")
pi_pmc_ids = dict(zip(pi_data["PI Name"], pi_data["PMC ID"]))

filtered_papers = []
total_pis_processed = 0

# Process research papers for each PI
for pi, author_id in pi_pmc_ids.items():
    if not author_id:
        continue

    page = 1
    has_more_pages = True

    while has_more_pages:
        params = {
            "db": "pubmed",
            "term": author_id,
            "retmax": 200,
            "retstart": (page - 1) * 200,
            "retmode": "xml"
        }

        try:
            response = requests.get(ncbi_esearch_url, params=params)
            time.sleep(1)

            if response.status_code == 200:
                root = ElementTree.fromstring(response.content)
                id_list = root.find("IdList")
                if id_list is not None:
                    pmids = [id_elem.text for id_elem in id_list.findall("Id")]

                    for pmid in pmids:
                        efetch_params = {
                            "db": "pubmed",
                            "id": pmid,
                            "retmode": "xml"
                        }
                        efetch_response = requests.get(ncbi_efetch_url, params=efetch_params)
                        time.sleep(1)

                        if efetch_response.status_code == 200:
                            efetch_root = ElementTree.fromstring(efetch_response.content)
                            article = efetch_root.find(".//PubmedArticle")

                            if article is not None:
                                title_elem = article.find(".//ArticleTitle")
                                abstract_elem = article.find(".//Abstract/AbstractText")
                                title = title_elem.text if title_elem is not None else 'No Title'
                                abstract = abstract_elem.text if abstract_elem is not None else ''

                                title_lower = title.lower()
                                abstract_lower = abstract.lower()
                                if (any(keyword in title_lower for keyword in fsrdc_keywords) or
                                    any(keyword in abstract_lower for keyword in fsrdc_keywords)):

                                    filtered_papers.append({
                                        "PI": pi,
                                        "Year": article.find(".//PubDate/Year").text if article.find(".//PubDate/Year") is not None else 'Unknown',
                                        "Title": title,
                                        "DOI": article.find(".//ELocationID[@EIdType='doi']").text if article.find(".//ELocationID[@EIdType='doi']") is not None else 'No DOI',
                                        "PMC ID": pmid
                                    })

                if len(pmids) < 200:
                    has_more_pages = False
                else:
                    page += 1
            else:
                break

        except Exception:
            break

    total_pis_processed += 1

# Final output
print(f"\n🔹 Processing complete!")
print(f"  Total PIs processed: {total_pis_processed}")
print(f"  Total matched papers: {len(filtered_papers)}")

df = pd.DataFrame(filtered_papers)
df.to_csv("pmc_api_ouput.csv", index=False)
print(f"Results saved to 'pmc_api_ouput.csv'")


# Data Processing

In [ ]:
# csv paths
csv_files = [
    'filtered_papers_openalex.csv',
    'filtered_papers_pmc.csv',
    'fsrdc_related_papers (1).csv',
    'pi_publications_fsrdc_coreapi (1).csv',
    'pi_publications_fsrdc_coreapi.csv',
    'fsrdc_keyword_matches.csv'
]


In [ ]:
import pandas as pd

# add files

# reformat all csvs
target_cols = ['pi', 'title', 'year', 'doi']
dfs = []
for file in csv_files:
    df = pd.read_csv(file)

    # Normalize column names to lowercase
    df.columns = df.columns.str.lower()

    # Filter columns only if they exist
    available_cols = [col for col in target_cols if col in df.columns]
    filtered_df = df[available_cols]

    dfs.append(filtered_df)

# combine all dfs into one & save
combined_df = pd.concat(dfs, ignore_index=True)
print(len(combined_df))

In [ ]:
# reformat year + get rid of invalid dois + filter duplicates
combined_df['year'] = pd.to_numeric(combined_df['year'], errors='coerce').astype('Int64')
combined_df = combined_df.dropna(subset=['year'])
combined_df = combined_df[combined_df['doi'].str.contains(r'^10\.\d{4,9}/.+', na=False)]
combined_df = combined_df.drop_duplicates(subset='doi').copy()
combined_df.to_csv('combined_output.csv', index=False)
combined_df = combined_df[combined_df['year'] >= 1990]
print(len(combined_df))

In [ ]:
# normalise data here
combined_df['title_clean'] = combined_df['title'].str.lower().str.strip()

# normalise data in the 2024 file
other_df = pd.read_excel('ResearchOutputs (1).xlsx')
other_df['title_clean'] = other_df['OutputTitle'].str.lower().str.strip()

# compare
combined_df = combined_df[~combined_df['title_clean'].isin(other_df['title_clean'])]
combined_df.to_csv('unique_entries.csv', index=False) # final unique
print(len(combined_df))

In [ ]:
!pip install fuzzywuzzy[speedup]
!pip install rapidfuzz
!pip install rapidfuzz

In [ ]:
# additionally erase roughly similar
from rapidfuzz import fuzz, process
from rapidfuzz.distance import Levenshtein
import numpy as np

threshold = 90 # random similarity threshold

# cleaned titles from excel to list
titles_1 = combined_df['title_clean'].tolist()
titles_2 = other_df['title_clean'].tolist()

score_matrix = process.cdist(titles_1, titles_2, scorer=fuzz.token_sort_ratio)
max_scores = np.max(score_matrix, axis=1)

# show top 20 similarity scores
print(max_scores[:20])
print("Highest match score:", max_scores.max())
print("Average match score:", max_scores.mean())

combined_df['is_unique'] = max_scores < threshold
unique_df = combined_df[combined_df['is_unique']]

unique_df.to_csv('unique_entries.csv', index=False) # final unique
print(len(unique_df))

# Graphing/Visualisation

Make Data Structure

In [ ]:
class Publication:
  def __init__(self, title, year, pi, doi, keywords):
     self.title = str(title)
     self.year = self.safe_int(year)
     self.pi = str(pi)
     self.doi = str(doi)
     self.keywords = str(keywords)

  def safe_int(self, value):
    try:
        return int(value)
    except ValueError:
        return None

  def __lt__(self, other):
    # sort by title (alphabetically)
    if self.title != other.title:
        return self.title < other.title
    return self.year < other.year

  def __repr__(self):
    # represent publication w title by default
    return f"Publication(title='{self.title}', year={self.year})"

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT # import keybert for keyword extraction
kw_model = KeyBERT() # initialize the model

In [ ]:
import pandas as pd

df2 = pd.read_csv('unique_entries.csv') # read the csv file

df2["title"] = df2["title"].astype(str).fillna("") # make sure all titles are strings + prevent the NaN exception
for index, row in df2.iterrows(): # iterate through each row in the dataframe
  title = row['title'] # get the title of the row from which we can extract keywords
  keywords = kw_model.extract_keywords(title, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=5) # extract top 5 keywords from the title
  formatted_keywords = "; ".join([kw[0] for kw in keywords]) # store the keywords as desired
  print(formatted_keywords)
  df2.loc[index, 'keywords'] = formatted_keywords # update cell

df2["year"] = df2["year"].fillna(0).astype(int).replace(0, "") # makes my year into a number because omg
df2.to_csv("data_with_keywords.csv", index=False)

In [ ]:
publications = []
for index, row in df2.iterrows():
    publication = Publication(row['title'], row['year'], row['pi'], row['doi'], row['keywords'])
    publications.append(publication)

In [ ]:
import networkx as nx # import library for making graph

G = nx.MultiGraph() # initialise graph
G.add_nodes_from(publications[:20]) # add publications as nodes

In [ ]:
edge_colours = {
    "title": "green",
    "year": "red",
    "pi": "blue",
    "keyword": "purple"
}

for publication in publications:
  for another_publication in publications: # compare every publication with every publication
    if publication is another_publication: # ignore if it's the same publication (doesn't need comparison])
      continue
    conditions = [ # check if title, year, pi or at least one keyword is alike
      (publication.title == another_publication.title, "title"),
      (publication.year == another_publication.year, "year"),
      (publication.pi == another_publication.pi, "pi"),
      (any(keyword in another_publication.keywords for keyword in publication.keywords.split("; ")), "keyword")
    ]

    for condition, colour_label in conditions:
      if condition: # if title, year, agency or at least one keyword is alike, add edge
        G.add_edge(publication, another_publication, colour=edge_colours[colour_label])

In [ ]:
# return connected components for a subgraph of a particular colour
def count_colored_connected_components(G, colour):
  # get connected nodes given colour
  subgraph = G.edge_subgraph([(u, v, k) for u, v, k, data in G.edges(data=True, keys=True) if data.get("colour") == colour]).copy()

  # include isolated nodes
  for node in G.nodes():
    # make sure that we're not counting None as a separate component (though this is probably useless)
    attribute_name = next((key for key, value in edge_colours.items() if value == colour), None)
    if attribute_name and getattr(node, attribute_name, None) is not None:
      if any(data.get("colour") != colour for _, _, data in G.edges(node, data=True)):
        subgraph.add_node(node)  # add isolated node


  return len(list(nx.connected_components(subgraph)))

In [ ]:
total_nodes = len(G.nodes())
total_edges = len(G.edges())
edges_green = len([(u, v) for u, v, data in G.edges(data=True) if data.get("colour") == "green"])
edges_red = len([(u, v) for u, v, data in G.edges(data=True) if data.get("colour") == "red"])
edges_blue = len([(u, v) for u, v, data in G.edges(data=True) if data.get("colour") == "blue"])
edges_purple = len([(u, v) for u, v, data in G.edges(data=True) if data.get("colour") == "purple"])
number_connected_components_green = count_colored_connected_components(G, "green")
number_connected_components_red = count_colored_connected_components(G, "red")
number_connected_components_blue = count_colored_connected_components(G, "blue")
number_connected_components_purple = count_colored_connected_components(G, "purple")


print(f"Total nodes: {total_nodes}")
print(f"Total edges: {total_edges}")
print(f"Edges with colour 'green': {edges_green}")
print(f"Edges with colour 'red': {edges_red}")
print(f"Edges with colour 'blue': {edges_blue}")
print(f"Edges with colour 'purple': {edges_purple}")
print(f"Number of connected components (green edges): {number_connected_components_green}")
print(f"Number of connected components (red edges): {number_connected_components_red}")
print(f"Number of connected components (blue edges): {number_connected_components_blue}")
print(f"Number of connected components (purple edges): {number_connected_components_purple}")

In [ ]:
import matplotlib.pyplot as plt
import random

# sample a fixed number of nodes
sample_size = 50
selected_colour = "red"

# sample random nodes for quick convenient plot
sampled_nodes = random.sample(list(G.nodes()), min(len(G.nodes()), sample_size))

# find all edges between the selected node that have the prespecified colour
filtered_edges = [
  (u, v) for u, v, data in G.edges(data=True)
  if data.get("colour") == selected_colour and u in sampled_nodes and v in sampled_nodes
]

# make a subgraph of these
sampled_subgraph = nx.Graph()
sampled_subgraph.add_nodes_from(sampled_nodes)
sampled_subgraph.add_edges_from(filtered_edges)

# add labels
node_labels = {node: f"{node.title[:20]}...\n({node.year})" for node in sampled_subgraph.nodes()} # cut title to make it prettier

# plot the subgraph
default_node_positions = nx.spring_layout(sampled_subgraph, k=0.5, seed=102)  # generate consistent layout, modify k to individual CCs closer together
plt.figure(figsize=(10, 10))
edge_colors = [G[u][v].get("colour", "black") for u, v in sampled_subgraph.edges()]
nx.draw(
  sampled_subgraph,
  pos=default_node_positions,
  with_labels=True,
  labels=node_labels,
  node_size=100,
  edge_color=selected_colour,
  font_size=8
)
plt.title(f"Subgraph of {sample_size} Random Nodes with {selected_colour.capitalize()} Edges")
plt.savefig("year_plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random

# sample a fixed number of nodes
sample_size = 5000
selected_colour = "green"

# sample random nodes for quick convenient plot
sampled_nodes = random.sample(list(G.nodes()), min(len(G.nodes()), sample_size))

# find all edges between the selected node that have the prespecified colour
filtered_edges = [
  (u, v) for u, v, data in G.edges(data=True)
  if data.get("colour") == selected_colour and u in sampled_nodes and v in sampled_nodes
]

# make a subgraph of these
sampled_subgraph = nx.Graph()
sampled_subgraph.add_nodes_from(sampled_nodes)
sampled_subgraph.add_edges_from(filtered_edges)

# add labels
node_labels = {node: f"{node.title[:20]}..." for node in sampled_subgraph.nodes()} # cut title to make it prettier

# plot the subgraph
default_node_positions = nx.spring_layout(sampled_subgraph, k=0.5, seed=102)  # generate consistent layout, modify k to individual CCs closer together
plt.figure(figsize=(10, 10))
edge_colors = [G[u][v].get("colour", "black") for u, v in sampled_subgraph.edges()]
nx.draw(
  sampled_subgraph,
  pos=default_node_positions,
  with_labels=True,
  labels=node_labels,
  node_size=100,
  edge_color=selected_colour,
  font_size=8
)
plt.title(f"Subgraph of {sample_size} Random Nodes with {selected_colour.capitalize()} Edges")
plt.savefig("title_plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random

# sample a fixed number of nodes
sample_size = 100
selected_colour = "blue"

# sample random nodes for quick convenient plot
sampled_nodes = random.sample(list(G.nodes()), min(len(G.nodes()), sample_size))

# find all edges between the selected node that have the prespecified colour
filtered_edges = [
  (u, v) for u, v, data in G.edges(data=True)
  if data.get("colour") == selected_colour and u in sampled_nodes and v in sampled_nodes
]

# make a subgraph of these
sampled_subgraph = nx.Graph()
sampled_subgraph.add_nodes_from(sampled_nodes)
sampled_subgraph.add_edges_from(filtered_edges)

# add labels
node_labels = {node: f"{node.title[:20]}...\n({node.pi})" for node in sampled_subgraph.nodes()} # cut title to make it prettier

# plot the subgraph
default_node_positions = nx.spring_layout(sampled_subgraph, k=0.9, seed=102)  # generate consistent layout, modify k to individual CCs closer together
plt.figure(figsize=(10, 10))
edge_colors = [G[u][v].get("colour", "black") for u, v in sampled_subgraph.edges()]
nx.draw(
  sampled_subgraph,
  pos=default_node_positions,
  with_labels=True,
  labels=node_labels,
  node_size=100,
  edge_color=selected_colour,
  font_size=8
)
plt.title(f"Subgraph of {sample_size} Random Nodes with {selected_colour.capitalize()} Edges")
plt.savefig("pi_plot.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import random

# sample a fixed number of nodes
sample_size = 150
selected_colour = "purple"

# sample random nodes for quick convenient plot
sampled_nodes = random.sample(list(G.nodes()), min(len(G.nodes()), sample_size))

# find all edges between the selected node that have the prespecified colour
filtered_edges = [
  (u, v) for u, v, data in G.edges(data=True)
  if data.get("colour") == selected_colour and u in sampled_nodes and v in sampled_nodes
]

# make a subgraph of these
sampled_subgraph = nx.Graph()
sampled_subgraph.add_nodes_from(sampled_nodes)
sampled_subgraph.add_edges_from(filtered_edges)

# add labels
node_labels = {node: f"{node.title[:20]}...\n{node.year}" for node in sampled_subgraph.nodes()} # cut title to make it prettier

# plot the subgraph
default_node_positions = nx.spring_layout(sampled_subgraph, k=0.5, seed=102)  # generate consistent layout, modify k to individual CCs closer together
plt.figure(figsize=(10, 10))
edge_colors = [G[u][v].get("colour", "black") for u, v in sampled_subgraph.edges()]
nx.draw(
  sampled_subgraph,
  pos=default_node_positions,
  with_labels=True,
  labels=node_labels,
  node_size=100,
  edge_color=selected_colour,
  font_size=8
)
plt.title(f"Subgraph of {sample_size} Random Nodes with {selected_colour.capitalize()} Edges")
plt.savefig("keywords_plot.png", dpi=300, bbox_inches='tight')
plt.show()

#Data analysis and Visualization

In [ ]:
import matplotlib.pyplot as plt

# get num of projects per year
projects_per_year = combined_df['year'].value_counts().sort_index()

# plt
plt.figure(figsize=(12, 6))
projects_per_year.plot(kind='bar')

plt.xlabel("Year")
plt.ylabel("Number of Projects")
plt.title("Number of Projects per Year")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("projects-over-time.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
peak_year = papers_by_year.idxmax()
peak_count = papers_by_year.max()
print(f"Peak year: {peak_year} ({peak_count} papers)")

In [ ]:
post_2015 = papers_by_year[papers_by_year.index >= 2015].sum()
pre_2015 = papers_by_year[papers_by_year.index < 2015].sum()
growth = (post_2015 - pre_2015) / pre_2015 * 100
print(f"Post-2015 papers: {post_2015} | Growth: {growth:.1f}%")

In [ ]:
rolling_avg = papers_by_year.rolling(window=3).mean()
rolling_avg.plot(figsize=(12, 5), color='green', label='3-Year Avg')
papers_by_year.plot(kind='line', alpha=0.3, label='Actual')
plt.legend()
plt.title("Smoothed Trend (3-Year Rolling Average)")
plt.show()

In [ ]:
import pandas as pd

# Load the csv file
df = pd.read_csv("unique_entries.csv")

# Check first few rows
print(df.head())

# Check basic info (data types, non-null counts)
print(df.info())

In [ ]:
#Top publishing PIs
import matplotlib.pyplot as plt
import seaborn as sns

top_pis= df.groupby('pi')['title'].count().sort_values(ascending=False).head(10)
print(top_pis)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_pis.values, y=top_pis.index, palette="viridis")
plt.title("Top 10 Publishing PIs")
plt.xlabel("Number of Publications")
plt.ylabel("Principal Investigator (PI)")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


# Create bigrams
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words='english')
X = vectorizer.fit_transform(df['title'].dropna().astype(str))
bigrams = zip(vectorizer.get_feature_names_out(), X.sum(axis=0).tolist()[0])
sorted_bigrams = sorted(bigrams, key=lambda x: x[1], reverse=True)[:15]

# Plot
bigram_labels, bigram_counts = zip(*sorted_bigrams)
plt.figure(figsize=(10,6))
sns.barplot(x=list(bigram_counts), y=list(bigram_labels), palette='crest')
plt.title("Top Bigrams in Research Titles")
plt.xlabel("Frequency")
plt.ylabel("Bigram")
plt.tight_layout()
plt.show()


# Descriptive Statistics & Visualization

In [ ]:
pd.options.display.float_format = '{:.0f}'.format  # No decimals
print(df['year'].describe())

In [ ]:
# Unique pis
print("Unique pis:", df['pi'].nunique())

In [ ]:
#Graph showing frequency of publications over time.
plt.figure(figsize=(10, 5))
plt.hist(df['year'], bins=range(2010, 2024), edgecolor='black', alpha=0.7)
plt.title("Distribution of FSRDC Papers by Year")
plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.xticks(range(2010, 2024))
plt.grid(axis='y', linestyle='--')
plt.show()

In [ ]:
#A box plot showing outliers in publication years.
plt.figure(figsize=(8, 4))
plt.boxplot(df['year'].dropna(), vert=False, patch_artist=True)
plt.title("Box Plot of Publication Years")
plt.xlabel("Year")
plt.yticks([])
plt.show()